In [ ]:
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, VitalSign
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
from fiber.utils import Timer
import pickle

import math
import os
import sys
from functools import reduce
from fiber.storage import yaml as fiberyaml
from fiber.utils import Timer


In [ ]:
#with 0 threshold 
Drug = pq.read_table('Parquets/Cases/Consolidated/with_0_threshold/Drug_Cases_Unsupervised_All.parquet').to_pandas()
Diagnosis = pq.read_table('Parquets/Cases/Consolidated/with_0_threshold/Diagnosis_Cases_Unsupervised_All.parquet').to_pandas()
VitalSign = pq.read_table('Parquets/Cases/Consolidated/with_0_threshold/VitalSign_Cases_Unsupervised_All.parquet').to_pandas()
Lab = pq.read_table('Parquets/Cases/Consolidated/with_0_threshold/LabValue_Cases_Unsupervised_All.parquet').to_pandas()
Proc = pq.read_table('Parquets/Cases/Consolidated/with_0_threshold/Procedure_Cases_Unsupervised_All.parquet').to_pandas()

Combined = pq.read_table('Parquets/Final/with_0_threshold/Merged_Cases_0.parquet').to_pandas()


In [ ]:
Combined= Combined.reindex(sorted(Combined.columns), axis=1)

In [ ]:
for col in Combined:
    print (col)

In [ ]:
Combined

In [ ]:
Diagnosis

In [ ]:
Drug

In [ ]:
VitalSign

In [ ]:
Lab

In [ ]:
Proc

In [ ]:
Case.loc[Case['Diagnosis__TDS__TIA'] > 0]

In [ ]:
# Reading cohort as dataframe

#Case = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Cases_HT_arterial_disease_unsupervised.pkl')

#Case = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Cases_HT_CR_unsupervised.pkl')
#Case = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Cases_HT_dementia_unsupervised.pkl')
#Case = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Cases_HT_heart_disease_unsupervised.pkl')
#Case = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Cases_HT_hypertensive_disease_unsupervised.pkl')
#Case = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Cases_HT_renal_unsupervised.pkl')

#Case

# Controls
#Control = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Controls_arterial_disease_unsupervised.pkl')
# Control = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Controls_Cerebrovascular_Diseases_unsupervised.pkl')
# Control = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Controls_dementia_unsupervised.pkl')
# Control = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Controls_Heart_Diseases__unsupervised.pkl')
# Control = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Controls_Hypertensive_Diseases__unsupervised.pkl')
# Control = pd.read_pickle('Unsupervised_Cohorts/Unsupervised_Cohorts_Final/Controls_renal__unsupervised.pkl')
 
#Control


In [ ]:
# Unsupervised Feature extraction - PIVOT config#

In [ ]:
# CONFIG Drugs
DEFAULT_PIVOT_CONFIG = { Drug(): {
'window':(-math.inf, 0),'pivot_table_kwargs': {
'columns': ['code'],
'aggfunc': {'code': 'count' }}, 'threshold': 0.5
    }
} 

In [ ]:
# CONFIG Diagnosis
DEFAULT_PIVOT_CONFIG = { Diagnosis(): {
'window':(-math.inf, 0), 'pivot_table_kwargs': {
'columns': ['code'],
'aggfunc': {'code': 'any'} }, 'threshold': 0.5
    }
} 

In [ ]:
# CONFIG VitalSigns
DEFAULT_PIVOT_CONFIG = { VitalSign(): {
'window':(-math.inf, 0),'pivot_table_kwargs': {
'columns': ['code'],
'aggfunc': {'numeric_value': ['min', 'median', 'max']}},'threshold': 0.1
    }
} 

In [ ]:
# CONFIG LabValue
DEFAULT_PIVOT_CONFIG = { LabValue(): {
'window':(-math.inf, 0), 'pivot_table_kwargs': {
'columns': ['test_name'],
'aggfunc': {'numeric_value': ['min', 'median', 'max']} }, 'threshold': 0.1
    }
} 

In [ ]:
DEFAULT_PIVOT_CONFIG = { Procedure(): {
'window':(-math.inf, 0), 'pivot_table_kwargs': {
'columns': ['code'],
'aggfunc': {'code' : 'count'} }, 'threshold': 0.1
    }
}


In [ ]:
#to try with small sample
Case = Case.sample(50) 
Case


In [ ]:
# running through entire cohort in batches and saving each batch as parquet
for limit in range (0, len(Case), 50):
    print("Begin of iteration: " +  str(limit))
    
    temp = Case[limit:(limit+50000)]
    p_condition = MRNs(temp) #how to create cohort from dataframe
    cohort = Cohort(p_condition)
    result = cohort.get_pivoted_features(pivot_config=DEFAULT_PIVOT_CONFIG)
    
    #result.to_parquet('Unsupervised_Features/Cases_Drugs' + str(limit))


In [ ]:
result